In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from itertools import combinations
from decimal import Decimal

pd.set_option('display.max_columns', None)

In [2]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [3]:
situacao_final = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/situacao_final.xlsx')).set_index('Ordem')
local_frac = pd.read_excel(caminho_absoluto('data/analise_curva_abc/local/datasets/local_apanha_frac.xlsx'))

In [210]:
armazenagens = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/datasets/armazenagens.xlsx'))
armazenagens_estoque = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/datasets/armazenagens_estoque.xlsx'))
armazenagens_estoque.fillna('-', inplace=True)

In [4]:
situacao_final.iloc[[0]]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,106011,NARIDRIN 12 HORAS FR C/30ML,A,A,A,279389.0,6507.0,930089.0,271.0,251.0,274.0,1490.0,164.0,1490.0,1030.9557,25.9243,3394.4854,5057,016-037-01-02,50,2,100,387,500,Não,29.913,Ponta De G,3236.0,pallet


In [5]:
local_frac.iloc[[0]]

,Ender.Fracionado,modulo,local
0,29.011,1,AB


In [38]:
armazenagens

,,Dep.,Apelido,Região,.1,Situação,.2,Função,Tipo Apanha Frac.,Altura,Num.Posto,Descr.Posto,Motivo Bloqueio,.3,Data Último Invent.,Sit.Último Invent.,Total Invent.
0,NaN,1,010-002-03-01,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE,NaN,NaT,Pendente,0
1,NaN,1,010-002-03-02,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE,NaN,NaT,Pendente,0
2,NaN,1,010-002-04-01,1,NaN,Uso,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,NaN,NaN,2023-10-30 10:41:19,Cancelado,3
3,NaN,1,010-002-04-02,1,NaN,Bloq.Invent.,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,INVENTARIO,NaN,2024-03-14 14:50:23,Pendente,7
4,NaN,1,010-002-05-01,1,NaN,Bloq.Invent.,NaN,Armazenagem,NaN,ALTURA MAIOR PADRAO,0,NaN,INVENTARIO,NaN,2024-03-13 08:27:55,Pendente,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,NaN,2,018-010-05-02,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE - NORMAL DEP 2,NaN,2024-03-14 14:20:59,Cancelado,1
2821,NaN,2,018-010-06-01,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE - NORMAL DEP 2,NaN,2024-03-14 14:20:59,Cancelado,1
2822,NaN,2,018-010-06-02,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE - NORMAL DEP 2,NaN,2024-03-14 14:20:59,Cancelado,1
2823,NaN,2,018-010-07-01,1,NaN,Inativo,NaN,Armazenagem,NaN,ALTURA PADRAO PALETE,0,NaN,.NÃO EXISTE - NORMAL DEP 2,NaN,2024-03-14 14:20:59,Cancelado,1


In [13]:
ruas_selec = ['010', '012', '014', '015', '016', '017', '018', '100']

colunas = ['Dep.',
            'Apelido',
            'Situação',
            'Motivo Bloqueio',
            ]

todas_armazenagens = armazenagens[colunas]
todas_armazenagens['Motivo Bloqueio'].fillna('-', inplace=True)

condicao = False
for rua in ruas_selec:
    condicao |= todas_armazenagens['Apelido'].str.startswith(rua)

todas_armazenagens = todas_armazenagens[condicao]

selecao_porta_pallets = (todas_armazenagens['Dep.'] == 1) & \
                    (todas_armazenagens['Situação'] != 'Inativo') & \
                    (~todas_armazenagens['Motivo Bloqueio'].str.contains('NÃO EXISTE', case=False)) & \
                    (~todas_armazenagens['Motivo Bloqueio'].str.contains('AGREGADO', case=False))

porta_pallets = todas_armazenagens[selecao_porta_pallets]
total_porta_pallets = len(porta_pallets)


selecao_porta_pallets = (todas_armazenagens['Dep.'] == 1) & \
                    (todas_armazenagens['Situação'] != 'Inativo') & \
                    (~todas_armazenagens['Motivo Bloqueio'].str.contains('NÃO EXISTE', case=False)) & \
                    (~todas_armazenagens['Motivo Bloqueio'].str.contains('AGREGADO', case=False))

porta_pallets = todas_armazenagens[selecao_porta_pallets]
total_porta_pallets = len(porta_pallets)

selecao_porta_pallets_bloq = (porta_pallets['Situação'] == 'Bloqueado')

porta_pallets_bloq = porta_pallets[selecao_porta_pallets_bloq]

total_porta_pallets_bloq = len(porta_pallets_bloq)

motivos_bloq = porta_pallets_bloq['Motivo Bloqueio'].value_counts()
motivos_bloq = pd.DataFrame(motivos_bloq).reset_index()
motivos_bloq['Motivo Bloqueio'] = motivos_bloq['Motivo Bloqueio'].str.lstrip('.')
motivos_bloq.rename(columns={'count':'N° Armazenagens'}, inplace=True)

selecao_porta_pallet_dif_bloq = ~porta_pallets['Apelido'].isin(porta_pallets_bloq['Apelido'])

porta_pallet_dif_bloq = porta_pallets[selecao_porta_pallet_dif_bloq]

total_porta_pallet_dif_bloq = len(porta_pallet_dif_bloq)

situacao_utilizaveis = porta_pallet_dif_bloq['Situação'].value_counts()
situacao_utilizaveis = pd.DataFrame(situacao_utilizaveis).reset_index()
situacao_utilizaveis['Situação'] = situacao_utilizaveis['Situação'].str.lstrip('.')
situacao_utilizaveis.rename(columns={'count':'Quantidade'}, inplace=True)

valores_situacao_utilizaveis = situacao_utilizaveis['Situação'].values

if 'Uso' in valores_situacao_utilizaveis:
  total_uso = situacao_utilizaveis['Quantidade'][situacao_utilizaveis['Situação'] == 'Uso'].iloc[0]
else:
  total_uso = 0

if 'Bloq.Invent.' in valores_situacao_utilizaveis:
  total_inv = situacao_utilizaveis['Quantidade'][situacao_utilizaveis['Situação'] == 'Bloq.Invent.'].iloc[0]
else:
  total_inv = 0
  
if 'lib' in valores_situacao_utilizaveis:
  total_lib = situacao_utilizaveis['Quantidade'][situacao_utilizaveis['Situação'] == 'lib'].iloc[0]
else:
  total_lib = 0
  
arm_uso = pd.DataFrame({'Situação' : ['Liberado', 'Em uso'],
                    'Quantidade' : [(total_inv + total_lib), (total_uso)]})

fig = px.pie(arm_uso, values='Quantidade', names='Situação', color='Situação', 
              color_discrete_map={'Em uso':'lightgrey',
                                  'Liberado':'mediumblue'})

situacao_utilizaveis_graf_pizza = fig.update_traces(textposition='outside', textinfo='percent+label')

C:\Users\Renan\AppData\Local\Temp\ipykernel_18192\1010731029.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
situacao_utilizaveis_graf_pizza

In [17]:
total_uso

1776

In [252]:
ruas_selec = ['017', '018']

condicao = False
for rua in ruas_selec:
    condicao |= armazenagens_estoque['Endereço'].str.startswith(rua)

arm_selec = armazenagens_estoque['Endereço'][condicao]

df = pd.DataFrame()

for endereco in armazenagens_estoque['Endereço']:

  if endereco in arm_selec.values:
    i_linha_atual = armazenagens_estoque['Endereço'][armazenagens_estoque['Endereço'] == endereco].index
    
    e_fd = armazenagens_estoque['Motivo Bloq.Ender.'].iloc[i_linha_atual[0] + 2].startswith('FD')
    
    if e_fd:
      tipo = 'FD'
    else:
      tipo = 'Outros'
    
    linha = pd.DataFrame({'End' : [endereco], 'Tipo': [tipo]})
    df = pd.concat([df, linha], ignore_index=True)
    
relacao_em_uso = pd.DataFrame(df['Tipo'].value_counts()).reset_index().rename(columns={'count' : 'Quantidade'})
relacao_em_uso

fig = px.pie(relacao_em_uso, values='Quantidade', names='Tipo', color='Tipo', 
              color_discrete_map={'FD':'lightgrey',
                                  'Outros':'mediumblue'})

situacao_em_uso = fig.update_traces(textposition='outside', textinfo='percent+label')
situacao_em_uso